In [ ]:
import pandas as pd
import re
import spacy
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Charger le modèle linguistique français de spaCy
nlp = spacy.load("fr_core_news_sm")

: 

In [ ]:
# Charger les données
DataFrame = pd.read_excel("/Users/charlescook/Desktop/Avis_bar.xlsx")

In [17]:
# Supprimer les lignes où il manque des avis ou des notes 
DataFrame = DataFrame.dropna(subset=["Avis", "Note"])
DataFrame["Note"] = pd.to_numeric(DataFrame["Note"], errors="coerce")


In [18]:
# Nettoyage des avis
def nettoyage_prealable(text):
    # Conversion en minuscule
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    # Retirer les espaces multiples
    text = re.sub(r"\s+", " ", text).strip()
    return text

DataFrame["Avis"] = DataFrame["Avis"].apply(nettoyage_prealable)

In [19]:
# Traitement avec spaCy : tokenisation, suppression des stopwords et lemmatisation
def spacy_process(text):
    doc = nlp(text) # Création du doc qui contient des tokens, des informations grammaticales sur le mot
    tokens = [
        token.lemma_  # Lemmatisation (transformationd des verbes en infinitif par exemple)
        for token in doc
        if not token.is_stop and not token.is_punct  # Retirer stopwords et ponctuation
    ]
    return tokens

DataFrame["Tokens"] = DataFrame["Avis"].apply(spacy_process)

In [20]:
# Reconstituer les avis nettoyés et lemmatisés
DataFrame["Cleaned_Avis"] = DataFrame["Tokens"].apply(lambda x: " ".join(x))

In [21]:
DataFrame.to_excel("/Users/charlescook/Desktop/resultats_modifies_bar.xlsx", index=False)

In [22]:
# Configurer TextBlob pour le français
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

# Charger le fichier Excel modifié
file_path = "/Users/charlescook/Desktop/resultats_modifies_bar.xlsx"  # Remplacez par le chemin correct
colonne = pd.read_excel(file_path)

In [23]:
# Fonction pour analyser les sentiments avec TextBlob
def analyze_sentiment(text): # en entrée on prend un chaâine de caractères
    blob = tb(text) # objet qui analyse le texte notamment la polarité et la subjectivité
    return blob.sentiment # renvoie un tuple contenant polarité et subjectivité
# Appliquer l'analyse de sentiment
colonne["Sentiment"] = colonne["Cleaned_Avis"].apply(lambda x: analyze_sentiment(x) if isinstance(x, str) else None) 

In [24]:
# Séparer la polarité et la subjectivité dans deux colonnes distinctes
colonne["Polarité"] = colonne["Sentiment"].apply(lambda x: x[0] if x else None)  # Polarité : entre -1 et 1
colonne["Subjectivité"] = colonne["Sentiment"].apply(lambda x: x[1] if x else None)  # Subjectivité : entre 0 et 1

# Supprimer la colonne intermédiaire "Sentiment"
colonne = colonne.drop(columns=["Sentiment"])

# Sauvegarder les résultats dans un nouveau fichier Excel
output_path = "/Users/charlescook/Desktop/résultats_avec_sentiments.xlsx"  # Chemin pour le fichier de sortie
colonne.to_excel(output_path, index=False)

In [39]:
import pycaret

In [40]:
from pycaret import clustering

In [ ]:
DataFrame = pd.read_excel("/Users/charlescook/Desktop/Avis_bar.xlsx")
ce = clustering.ClusteringExperiment()

ce.setup(data=DataFrame,text_features=['Avis'])

In [ ]:
km = ce.create_model('kmeans',num_clusters=2,random_state=123)

In [ ]:
# Charger les données
DataFrame = pd.read_excel("/Users/charlescook/Desktop/Avis_bar.xlsx")

# Sélectionner un échantillon de 10 avis
sampled_df = DataFrame.sample(n=10, random_state=123)

# Initialiser l'expérience de clustering sur l'échantillon
ce = clustering.ClusteringExperiment()

# Configurer les données
ce.setup(data=sampled_df, text_features=['Avis'])

# Créer le modèle de clustering K-means
km = ce.create_model('kmeans', num_clusters=2, random_state=123)

# Prédire les clusters pour l'échantillon
cluster_labels = ce.predict_model(km)

# Ajouter la colonne des clusters au DataFrame échantillonné
sampled_df['Cluster'] = cluster_labels['Cluster']

# Sauvegarder l'échantillon avec la colonne des clusters dans un nouveau fichier Excel
sampled_df.to_excel("/Users/charlescook/Desktop/Avis_bar_sampled_with_clusters.xlsx", index=False)